# 人脸关键点检测及节点数据细节

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 定义可视化图像函数
def look_img(img):
    '''opencv读入图像格式为BGR，matplotlib可视化格式为RGB，因此需将BGR转RGB'''
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_RGB)
    plt.show()

In [ ]:
# 导入人脸关键点检测模型
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=5,# 最大人脸数量
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
# 导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils # 画关键点
drawing_spec = mp_drawing.DrawingSpec(
    thickness=3,# 线宽 
    circle_radius=5,# 点半径
    color=(223,155,6)
)

In [ ]:
# 读取图像
img = cv2.imread('../images/A-0.jpeg')
look_img(img)

In [ ]:
# 将BGR图像转RGB
img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 将RGB图像输入模型，获取预测结果
results = face_mesh.process(img_RGB)

if results.multi_face_landmarks: # 判断是否检测到人脸
    for face_landmarks in results.multi_face_landmarks:# 遍历每一张人脸
        mp_drawing.draw_landmarks(
            image=img,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=drawing_spec,
            connection_drawing_spec=drawing_spec
        )
else:
    print('No face detected!')
    
look_img(img)

In [ ]:
# 查看脸轮廓关键点之间的链接关系
mp_face_mesh.FACEMESH_CONTOURS

In [ ]:
# 查看人脸的所有关键点坐标
# 注意：这是一个很奇怪的数据结构
# for face_landmarks in results.multi_face_landmarks:
#     print(face_landmarks.landmark)
    
results.multi_face_landmarks

In [ ]:
# 查看人脸关键点数量
len(mp_face_mesh.FACEMESH_CONTOURS)

In [ ]:
# 查看人脸的数量
len(results.multi_face_landmarks)

获取坐标值

In [ ]:
# 缩引为0的脸的关键点坐标
results.multi_face_landmarks[0]

In [ ]:
# 获取缩引为0的脸的454号关键点坐标
results.multi_face_landmarks[0].landmark[454]


In [ ]:
# 获取缩引为0的脸的454号关键点的x坐标
results.multi_face_landmarks[0].landmark[454].x

将相对坐标转为绝对像素坐标实现可视化(454号关键点)

In [ ]:
# 获取图像的宽高
img_width, img_height = img.shape[1], img.shape[0]
print(img_width,img_height)

In [ ]:
# 将相对坐标转为绝对像素坐标
x = int(results.multi_face_landmarks[0].landmark[454].x * img_width)
y = int(results.multi_face_landmarks[0].landmark[454].y * img_height)
print(x,y)

In [ ]:
# 画出关键点
# circle函数是画圆的函数
img = cv2.circle(img, (x,y), 10, (0,0,255), -1)
look_img(img)

In [ ]:
# 如法炮制画出缩引为1的脸的苹果机（123和352序号）
x = int(results.multi_face_landmarks[1].landmark[123].x * img_width)
y = int(results.multi_face_landmarks[1].landmark[123].y * img_height)
img = cv2.circle(img, (x,y), 20, (255,0,0), -1)

x = int(results.multi_face_landmarks[1].landmark[352].x * img_width)
y = int(results.multi_face_landmarks[1].landmark[352].y * img_height)
img = cv2.circle(img, (x,y), 20, (255,0,0), -1)

look_img(img)



In [ ]:
# 在3D空间中画出缩引为0的人脸
mp_drawing.plot_landmarks(results.multi_face_landmarks[0], mp_face_mesh.FACEMESH_CONTOURS)

In [ ]:
# 在3D空间中画出缩引为0的人脸
mp_drawing.plot_landmarks(results.multi_face_landmarks[0], mp_face_mesh.FACEMESH_TESSELATION)

In [ ]:
# 在3D空间中画出缩引为0的人脸
mp_drawing.plot_landmarks(results.multi_face_landmarks[0], mp_face_mesh.FACEMESH_IRISES)

利用Open3D实现交互式可视化

In [ ]:
# 以缩引为0的人为例
# 获取关键点坐标
coords = np.array(results.multi_face_landmarks[0].landmark)

In [ ]:
len(coords)

In [ ]:
# 注意，这个coords结构很诡异，需要进一步处理
print(coords)

In [ ]:
coords[1].x

In [ ]:
# 在不用耗时循环的条件下，汇总所有点的xyz坐标
def get_x(each):
    return each.x
def get_y(each):
    return each.y
def get_z(each):
    return each.z

# 分别获取所有关键点的xyz坐标
points_x = np.array(list(map(get_x, coords)))
points_y = np.array(list(map(get_y, coords)))
points_z = np.array(list(map(get_z, coords)))

# 将xyz坐标合并
points = np.vstack([points_x, points_y, points_z]).T

# 查看points的形状
print(points.shape)

In [ ]:
print(points)

In [ ]:
# 使用Open3D可视化（不推荐，open3d与jupyter存在兼容性问题，容易导致jupyter无法正常打开）
# import cv2
# import open3d as o3d
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)
# o3d.visualization.draw_geometries([pcd])